#### <center> The prupose of this notebook is to show how </center>
# <center> 🧬🧬🧬 Genetic Algorithms (GA's) 🧬🧬🧬 </center>
### <center> can do Feature Selection </center>

#### The workflow wil be the following:

1. Create a baseline model:
 - I chose a quick classifier, because GA's does a lot of iterations,
 - So by using ALL features, accuracy is computed (TPS's objective)
2. Do Feature Selection with GA's:
 - This is where GA's will find the best features subset of all 286 features,
 - Making an efficient model, and with that, having better accuracy

##### NOTE: The following results will be dependent on your choice of estimator, I chose famous Naives Bayes, but feel free to experiment with your own :)

#### <center> Lastly, if you find this useful, you might want to upvote </center>
# <center> Thank you for viewing </center>
### <center> On to GA's power now! 🧬🧬🧬 </center>

In [ ]:
%reset -sf

from gc import collect
collect(0)  # there are here because I generally hit the Run All button, to make sure everything works
collect(1)
collect(2)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# Installing Inte Xeon scikit accelerator
!pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()

# Data

In [ ]:
# Read data

from pandas import read_csv
from pathlib import Path

path = Path('/kaggle/input/tabular-playground-series-feb-2022')
#list(path.glob('*'))

train = read_csv(path / 'train.csv', )
train.info()

In [ ]:
# Reduce memory

from numpy import float64, float32, int64, int32, dtype

map_dtypes = {
    'int': dtype(int64),
    'float': dtype(float32)
    }

for col in train:
    if train[col].dtype == dtype(int64):
        train[col] = train[col].astype(int32)
    if train[col].dtype == dtype(float64):
        train[col] = train[col].astype(float32)
        
train.info()

In [ ]:
# Remove duplicates
# Why do we have duplicates...

train = train.drop(['row_id'], axis=1).drop_duplicates()

In [ ]:
# Map string y to num

y = train['target']
map_helper = dict(list(zip(
    y.unique(),
    range(len(y.unique()))
)))

y = y.map(map_helper)

X = train.drop(['target'], axis=1)

# Split

In [ ]:
# Split

from random import seed
seed(16)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

# Preprocessing

In [ ]:
# Prepr

from pandas import DataFrame

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train = DataFrame(scaler.fit_transform(X_train))
X_test = DataFrame(scaler.transform(X_test))
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Base

In [ ]:
# Baseline

from sklearn.naive_bayes import GaussianNB  # use this for quick iteration, or your own
est = GaussianNB()

from sklearn.pipeline import make_pipeline
pip = make_pipeline(est)
_ = pip.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error, f1_score, classification_report, accuracy_score
#print(classification_report(pip.predict(X_test), y_test))
f'Simple Naive Bayes Accuracy score is: {accuracy_score(pip.predict(X_test), y_test)*100:.1f} %'

## INTERPRETATION

### 1. Using all features leads to an Inefficient model,
### 2. As the number of features increase (relative to rows), models struggles for accuracy,
### 3. This what happens when, i.e. you have high collinearity with Lasso.

# Genetic Algorithms

In [ ]:
from deap import base, creator, tools, algorithms
from random import randint, choice, choices
from numpy import array
from pandas import DataFrame, Series, concat

toolbox = base.Toolbox()
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

def gen_genes(X_train):
    num_cols = X_train.shape[1]
    choices = [False, True]
    cols = [choice(choices) for _ in range(num_cols)]
    return cols

toolbox.register("genes", gen_genes, X_train)

toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.genes)

toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)  # needs n later

def eval_func(individual):
    """Fit and Evaluate for some criteria"""
    pip.fit(X_train.iloc[:, individual], y_train)
    return accuracy_score(pip.predict(X_test.iloc[:, individual]), y_test),

toolbox.register("evaluate", eval_func)

toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)

In [ ]:
from numpy import mean, array, max as np_max

POPULATION_SIZE = 50
MAX_GENERATIONS = 30
HALL_OF_FAME_SIZE = 5

P_CROSSOVER = 0.5
P_MUTATION = 0.1

# Genetic Algorithm flow:
population = toolbox.populationCreator(POPULATION_SIZE)
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", mean)
stats.register("max", np_max)

population, logbook = algorithms.eaSimple(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                    ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=True)

#minFitnessValues, meanFitnessValues = logbook.select("min", "avg")

# print best solution found:
best = hof.items[0]
#print("-- Best Ever Individual = ", array(best))
#print("-- Best Ever Individual = ", best)
#print("-- Best Ever Fitness = ", best.fitness.values[0])
f'Optimal Features Naive Bayes Accuracy score is now: {best.fitness.values[0]*100:.1f} %'

## INTERPRETATION

### Just by using some columns, we get an increase in accuracy!

## <center> 💪💪💪 TIP 💪💪💪</center>
## <center> Experiment with all GA's parameters, for example, for more insights, increase the number of population and/or generations </center>

In [ ]:
print(f'Original number of Features {X_train.shape[1]}')
print(f'Optimal Selected number of Features {X_train.iloc[:, best].shape[1]}')

In [ ]:
array(best)

In [ ]:
# ~ END ~